In [1]:
!pip install tflite-model-maker  &> /dev/null
!pip install tensorflow-datasets -U  &> /dev/null

In [2]:
import tensorflow_datasets as tfds
from tflite_model_maker import model_spec
from tflite_model_maker import image_classifier
from tflite_model_maker import ImageClassifierDataLoader

In [3]:
def get_dataloader(ds, size, labels):
  ds = ds.map(lambda a: (a['image'], a['label']))
  return ImageClassifierDataLoader(ds, size, labels)

In [15]:
batch_size = 16
img_height = 224
img_width = 224
#SPLITS = ['train[0%:80%]', 'train[80%:90%]', 'train[90%:100%]']
SPLITS = ['train', 'test', 'validation']
(ds_train, ds_validation, ds_test), ds_info = tfds.load(
    name = 'beans', 
    split = SPLITS,
    with_info = True)
label_names = ds_info.features['label'].names
size = ds_info.splits['train'].num_examples
val_size = ds_info.splits['validation'].num_examples
test_size = ds_info.splits['test'].num_examples
train_data = get_dataloader(ds_train, size, label_names)
validation_data = get_dataloader(ds_validation, val_size, label_names)
test_data = get_dataloader(ds_test, test_size, label_names)

In [16]:
model_handle = 'https://tfhub.dev/google/cropnet/feature_vector/cassava_disease_V1/1'
cassava_spec = model_spec.ImageModelSpec(uri=model_handle)

In [17]:
modelMM = image_classifier.create(train_data, model_spec=cassava_spec, 
                                  epochs=5, 
                                  train_whole_model=True, 
                                  validation_data=validation_data)

INFO:tensorflow:Retraining the models...


INFO:tensorflow:Retraining the models...


Model: "sequential_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
hub_keras_layer_v1v2_3 (HubK (None, 1280)              4234118   
_________________________________________________________________
dropout_3 (Dropout)          (None, 1280)              0         
_________________________________________________________________
dense_3 (Dense)              (None, 3)                 3843      
Total params: 4,237,961
Trainable params: 4,213,561
Non-trainable params: 24,400
_________________________________________________________________
None
Epoch 1/5
32/32 [==============================] - 19s 345ms/step - loss: 1.1238 - accuracy: 0.6703 - val_loss: 1.1379 - val_accuracy: 0.8438
Epoch 2/5
32/32 [==============================] - 9s 275ms/step - loss: 0.6137 - accuracy: 0.9200 - val_loss: 0.8496 - val_accuracy: 0.8828
Epoch 3/5
32/32 [==============================] - 8s 265ms/step - loss: 0.4311 -

In [18]:
modelMM.evaluate(validation_data)

4/4 [==============================] - 1s 179ms/step - loss: 0.4174 - accuracy: 0.9453


[0.4173751473426819, 0.9453125]

In [ ]:
modelMM.export(export_dir='./cropmodel/', tflite_filename='cassava_model.tflite')

INFO:tensorflow:Assets written to: /tmp/tmp5x8bbniy/assets


INFO:tensorflow:Assets written to: /tmp/tmp5x8bbniy/assets
